In [5]:
import numpy as np
import pandas as pd
import math

# Set random seed for reproducibility
np.random.seed(42)

# Number of synthetic samples
num_samples = 1000

# Synthetic RTL and signal names
rtl_modules = [f"module_{i}.v" for i in range(num_samples)]
signal_names = [f"signal_{i}" for i in range(num_samples)]

# Basic parameters
fan_in = np.random.randint(2, 9, size=num_samples)              # Fan-in between 2 and 8
gate_count = np.random.randint(10, 1001, size=num_samples)        # Gate count between 10 and 1000
fan_out = np.random.randint(1, 21, size=num_samples)              # Fan-out between 1 and 20
num_mux = np.random.randint(1, 11, size=num_samples)              # Multiplexers between 1 and 10

# Additional parameters
arith_ops = np.random.randint(0, 21, size=num_samples)             # Arithmetic operations between 0 and 20
capacitance = np.random.uniform(10, 1000, size=num_samples)         # Capacitance in fF
net_length = np.random.uniform(5, 200, size=num_samples)            # Net length in µm
bitwidth = np.random.randint(1, 65, size=num_samples)              # Bitwidth between 1 and 64

# Compute synthetic logic depth:
logic_depth = []
for i in range(num_samples):
    b = fan_in[i]
    n = gate_count[i]
    base_depth = math.ceil(math.log(n * (b - 1) + 1, b))
    extra_depth = round(fan_out[i] / 10) + round(num_mux[i] / 5) + round(arith_ops[i] / 5)
    total_depth = base_depth + extra_depth
    logic_depth.append(total_depth)

# Compute synthetic timing delay (in ns)
base_delay_per_level = np.random.uniform(0.1, 0.5, size=num_samples)
additional_delay = (
    fan_out * 0.01 +
    num_mux * 0.05 +
    arith_ops * 0.02 +
    (capacitance / 1000) * 0.1 +
    (net_length / 200) * 0.1 +
    (bitwidth / 64) * 0.05
)
timing_delay = np.array(logic_depth) * (base_delay_per_level + additional_delay)
timing_delay = np.clip(timing_delay, 0, None)

# Create DataFrame
df = pd.DataFrame({
    "RTL Module": rtl_modules,
    "Signal Name": signal_names,
    "Fan-in": fan_in,
    "Gate Count": gate_count,
    "Fan-out": fan_out,
    "Num Muxes": num_mux,
    "Arith Ops": arith_ops,
    "Capacitance (fF)": capacitance,
    "Net Length (um)": net_length,
    "Bitwidth": bitwidth,
    "Logic Depth": logic_depth,
    "Timing Delay (ns)": timing_delay
})

output_csv = "enhanced_synthetic_dataset_full.csv"
df.to_csv(output_csv, index=False)
print(f"Enhanced synthetic dataset generated and saved to {output_csv}")


Enhanced synthetic dataset generated and saved to enhanced_synthetic_dataset_full.csv


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Load the enhanced synthetic dataset
df = pd.read_csv("enhanced_synthetic_dataset_full.csv")

# Features: use all the synthetic parameters
features = ["Fan-in", "Gate Count", "Fan-out", "Num Muxes", "Arith Ops",
            "Capacitance (fF)", "Net Length (um)", "Bitwidth"]

X = df[features]
# Target: Logic Depth (the final computed depth)
y = df["Logic Depth"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Regressor
model = RandomForestRegressor(n_estimators=200, max_depth=12, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error on test set:", mae)

# Example Prediction:
# Suppose a new RTL signal has these features:
# Fan-in = 4, Gate Count = 500, Fan-out = 10, Num Muxes = 3,
# Arith Ops = 5, Capacitance = 300 fF, Net Length = 50 um, Bitwidth = 16.
sample_features = np.array([[4, 500, 10, 3, 5, 300, 50, 16]])
predicted_depth = model.predict(sample_features)
print("Predicted combinational logic depth for the sample features:", predicted_depth[0])


Mean Absolute Error on test set: 0.5436199145340038
Predicted combinational logic depth for the sample features: 8.264528728660737


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

DATASET_FILE = "enhanced_synthetic_dataset_full.csv"

def load_and_train_model(dataset_file):
    df = pd.read_csv(dataset_file)

    # Define feature columns and targets
    features = ["Fan-in", "Gate Count", "Fan-out", "Num Muxes", "Arith Ops",
                "Capacitance (fF)", "Net Length (um)", "Bitwidth"]
    targets = ["Logic Depth", "Timing Delay (ns)"]

    X = df[features]
    y = df[targets]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestRegressor(n_estimators=200, max_depth=12, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    print("Mean Absolute Error on test set:", mae)

    return model

def predict_synthesis_report(model):
    print("\nEnter the following features for the new RTL signal:")
    try:
        fan_in = float(input("Fan-in (e.g., 4): "))
        gate_count = float(input("Gate Count (e.g., 500): "))
        fan_out = float(input("Fan-out (e.g., 10): "))
        num_mux = float(input("Number of Multiplexers (e.g., 3): "))
        arith_ops = float(input("Number of Arithmetic Operations (e.g., 5): "))
        capacitance = float(input("Capacitance (fF) (e.g., 300): "))
        net_length = float(input("Net Length (um) (e.g., 50): "))
        bitwidth = float(input("Bitwidth (e.g., 16): "))
    except Exception as e:
        print("Error reading input:", e)
        return

    features_input = np.array([fan_in, gate_count, fan_out, num_mux, arith_ops, capacitance, net_length, bitwidth]).reshape(1, -1)
    prediction = model.predict(features_input)
    predicted_logic_depth = prediction[0][0]
    predicted_timing_delay = prediction[0][1]

    try:
        clock_period = float(input("Enter desired clock period (ns) (e.g., 10): "))
    except Exception as e:
        print("Error reading clock period:", e)
        return

    slack = clock_period - predicted_timing_delay

    print("\n--- Synthesis Report ---")
    print(f"Predicted combinational logic depth: {predicted_logic_depth:.2f} levels")
    print(f"Predicted critical path delay: {predicted_timing_delay:.2f} ns")
    print(f"Clock period: {clock_period:.2f} ns")
    print(f"Estimated timing slack: {slack:.2f} ns")
    print("------------------------\n")

def main():
    print("Loading dataset and training ML model for synthesis prediction...")
    model = load_and_train_model(DATASET_FILE)
    print("Model training complete.")

    predict_synthesis_report(model)

if __name__ == "__main__":
    main()


Loading dataset and training ML model for synthesis prediction...
Mean Absolute Error on test set: 0.9778725941606498
Model training complete.

Enter the following features for the new RTL signal:


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-a3496c1aa6a8>", line 83, in <cell line: 0>
    main()
  File "<ipython-input-4-a3496c1aa6a8>", line 80, in main
    predict_synthesis_report(model)
  File "<ipython-input-4-a3496c1aa6a8>", line 40, in predict_synthesis_report
    fan_in = float(input("Fan-in (e.g., 4): "))
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 1177, in raw_input
    return self._input_request(
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 1219, in _input_request
    raise KeyboardInterrupt("Interrupted by user") from None
KeyboardInterrupt: Interrupted by user

During handling of the above exception, another exception occurred:

Traceback (most recent c

TypeError: object of type 'NoneType' has no len()